In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run_test

import ecc
import helper
import tx
import script

In [11]:
from tx import Tx
from io import BytesIO

stream = BytesIO(bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600'))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [13]:
from helper import double_sha256

blob = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
d256 = double_sha256(blob)
z = int.from_bytes(d256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [12]:
from ecc import S256Point, Signature

sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


### Exercise 1

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest::test_sig_hash
```

In [22]:
# Exercise 1

reload(tx)
run_test(tx.TxTest('test_sig_hash'))

.
----------------------------------------------------------------------
Ran 1 test in 0.121s

OK


### Exercise 2

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest::test_verify_input
```

In [15]:
# Exercise 2

reload(tx)
run_test(tx.TxTest('test_verify_input'))

E
ERROR: test_verify_input (tx.TxTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jimmy/programming-bitcoin/code-ch07/tx.py", line 400, in test_verify_input
    self.assertTrue(tx.verify_input(0))
  File "/home/jimmy/programming-bitcoin/code-ch07/tx.py", line 110, in verify_input
    raise NotImplementedError
NotImplementedError

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


In [24]:
from helper import decode_base58, p2pkh_script
from script import Script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_ins.append(TxIn(prev_tx, prev_index, b'', 0xffffffff))
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
tx_outs.append(TxOut(amount=change_amount, script_pubkey=change_script))
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
tx_outs.append(TxOut(amount=target_amount, script_pubkey=target_script))
transaction = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(transaction)

version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13

tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG 
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG 

locktime: 0



In [25]:
from ecc import PrivateKey
from helper import SIGHASH_ALL

hash_type = SIGHASH_ALL
z = transaction.sig_hash(0, hash_type)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + hash_type.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

010000000199a24308080ab26e6fb65c4eccfadf76749bb5bfa8cb08f291320b3c21e56f0d0d0000006a47304402207de4c54c5dffefd8cb98fe3fd86faf7411a993b2e76d19d19bc7ac025ebf575602207b5d02bd7e9606e0cfc7cbba1a87dcff19ea2f7ab9af9aaf885d921aec413273012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67ffffffff02408af701000000001976a914d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f88ac80969800000000001976a914507b27411ccf7f16f10297de6cef3f291623eddf88ac00000000


In [28]:
from ecc import PrivateKey

private_key = PrivateKey(secret=90210)
print(private_key.point.address(testnet=True))

mqNK1JUujDXeufN9bDVKtzzvriqjnZLxHU


### Exercise 3

#### Make [this test](/edit/code-ch07/tx.py) pass
```
tx.py:TxTest:test_sign_input
```

In [26]:
# Exercise 3

reload(tx)
run_test(tx.TxTest('test_sign_input'))

E
ERROR: test_sign_input (tx.TxTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jimmy/programming-bitcoin/code-ch07/tx.py", line 454, in test_sign_input
    self.assertTrue(tx.sign_input(0, private_key, SIGHASH_ALL))
  File "/home/jimmy/programming-bitcoin/code-ch07/tx.py", line 156, in sign_input
    return self.verify_input(input_index)
  File "/home/jimmy/programming-bitcoin/code-ch07/tx.py", line 139, in verify_input
    raise NotImplementedError
NotImplementedError

----------------------------------------------------------------------
Ran 1 test in 0.974s

FAILED (errors=1)


### Exercise 4

#### Send 40% of the tBTC you've received to this address

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`

#### Go here to send your transaction: https://live.blockcypher.com/btc-testnet/pushtx/

#### You can get some free testnet coins at: https://testnet.coinfaucet.eu/en/

In [5]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts, 1 of the TxOuts should be back to your address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2


RuntimeError: Output is not something you can spend with this private key. Check that the prev_tx and prev_index are correct

### Exercise 5

#### Get some more testnet coins and spend both outputs (one from your change address and one from the testnet faucet) to

`mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`


In [6]:
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'

0100000002f48a065c1236ddf8033d2a1bc0ada4bca9da6adf4a5c1fb11edfaddd2efecb89000000006a47304402205b163c96216653e000c43521970cba3e1f115affb8935aeabc2e44ca2766497e0220578a2e8150b7663ef01d6c4f34d7798d357d850d93049a7b63aeba8be81fb4bd012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55befffffffff45ca5fc10cdf70aa347fcec838e97e021e88ed5d91133e0705fd904139e0619000000006b483045022100c0e7f756d07f262255bf96b00770d6b6bfa72f85376d9f0f07fc56386e5b206a02201d34ef0338726d7cc26205dea66e657b47011e55f67735f9bd3e869ec3e75ef8012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff01c040310a000000001976a9149e153ce5de3689901301309b9e8d804ab4804d4b88ac00000000


### Exercise 4

#### 4.1. Find the hash160 of the RedeemScript
```
5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [7]:
# Exercise 4.1

from helper import hash160

hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'

# bytes.fromhex script
redeem_script = bytes.fromhex(hex_redeem_script)
# hash160 result
h160 = hash160(redeem_script)
# hex() to display
print(h160.hex())

74d691da1574e6b3c192ecfb52cc8984ee7b6c56


In [8]:
# P2SH address construction example

from helper import encode_base58_checksum

print(encode_base58_checksum(b'\x05'+bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


### Exercise 5

#### 5.1. Make [these tests](/edit/session5/helper.py) pass
```
helper.py:HelperTest:test_p2pkh_address
helper.py:HelperTest:test_p2sh_address
```

In [9]:
# Exercise 5.1

reload(helper)
run_test(helper.HelperTest('test_p2pkh_address'))
run_test(helper.HelperTest('test_p2sh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [10]:
# z for p2sh example

from helper import double_sha256

sha = double_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
                    
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [11]:
# p2sh verification example

from ecc import S256Point, Signature
from helper import double_sha256

sha = double_sha256(bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))
z = int.from_bytes(sha, 'big')
point = S256Point.parse(bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
sig = Signature.parse(bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937'))
print(point.verify(z, sig))

True


### Exercise 6

#### 6.1. Validate the second signature of the first input

```
0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000
```

The sec pubkey of the second signature is:
```
03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71
```

The der signature of the second signature is:
```
3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022
```

The redeemScript is:
```
475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae
```

In [12]:
# Exercise 6.1

from io import BytesIO
from ecc import S256Point, Signature
from helper import double_sha256, int_to_little_endian
from script import Script
from tx import Tx, SIGHASH_ALL

hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb71'
hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022'
hex_redeem_script = '5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae'
sec = bytes.fromhex(hex_sec)
der = bytes.fromhex(hex_der)
redeem_script = bytes.fromhex(hex_redeem_script)

hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402201475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c568700000000'
stream = BytesIO(bytes.fromhex(hex_tx))

# parse the S256Point and Signature
point = S256Point.parse(sec)
sig = Signature.parse(der)
# parse the Tx
t = Tx.parse(stream)
# change the first input's scriptSig to redeemScript (use Script.parse on the redeemScript)
t.tx_ins[0].script_sig = Script.parse(redeem_script)
# get the serialization
ser = t.serialize()
# add the sighash (4 bytes, little-endian of SIGHASH_ALL)
ser += int_to_little_endian(SIGHASH_ALL, 4)
# double_sha256 the result
to_sign = double_sha256(ser)
# this interpreted is a big-endian number is your z
z = int.from_bytes(to_sign, 'big')
# now verify the signature using point.verify
print(point.verify(z, sig))

True
